# Fine Tuning of the convolutional network

In [1]:
%pylab
%matplotlib inline

import glob
import os
import mne
CORPORA_PATH = "~/corpora/sets"

file_path = os.path.expanduser(CORPORA_PATH)
files = glob.glob(os.path.join(file_path, "*.set"))

def normalize_subject(X):
    mean = X.mean(axis=(0, 2)).reshape(-1, 1)
    std = X.std(axis=(0, 2)).reshape(-1, 1)
    return (X - mean) / std

def load_data(filename, normalize=True):
    data_mne = mne.io.read_raw_eeglab(filename, preload=True, event_id={"0": 1, "1": 2})
    data_mne.filter(0, 20)
    events = mne.find_events(data_mne)
    epochs = mne.Epochs(
        data_mne, events,
        baseline=(None, 0), tmin=-0.1, tmax=0.7)

    epochs.load_data()
    
    ch_names = epochs.ch_names
    
    X = epochs.get_data()[:, :-1]
    y = (events[:, 2] == 2).astype('float')

    if len(events) != len(epochs):
        raise ValueError("Epochs events mismatch")
    if normalize: 
        X = normalize_subject(X)
    X = X[..., np.newaxis]
    
    return X, y 


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from keras.models import load_model
channels = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4', 'STI 014']

model = load_model("models/model.h5")

Using TensorFlow backend.
/home/ubuntu/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
X, y = load_data(files[143])

Reading /home/ubuntu/corpora/sets/PruebasMuseo_7488001.fdt
Reading 0 ... 63231  =      0.000 ...   493.992 secs...
Setting up low-pass filter at 20 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 169 samples (1.320 sec) selected
1800 events found
Events id: [1 2]
1800 matching events found
0 projection items activated
Loading data for 1800 events and 104 original time points ...
0 bad epochs dropped


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.1, stratify=y)

In [5]:


from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

def get_metrics(model, X_test, y_test):
    y_pred = model.predict_classes(X_test)
    y_prob = model.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    accuracy = accuracy_score(y_test, y_pred)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall, 
        "roc_auc": auc
    }
    
    
get_metrics(model, X_test, y_test)

{'accuracy': 0.53888888888888886,
 'precision': 0.22680412371134021,
 'recall': 0.73333333333333328,
 'roc_auc': 0.69155555555555548}

In [6]:
model.layers

Let's fix the first two convolutional layers

In [7]:

for i in range(4):
    model.layers[i].trainable = False



model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='rmsprop', 
              metrics=['accuracy']) # reporting the accuracy
[(l, "Trainable: {}".format(l.trainable)) for l in model.layers]

[(<keras.layers.convolutional.Conv2D at 0x7f3963894c18>, 'Trainable: False'),
 (<keras.layers.convolutional.Conv2D at 0x7f3963894f28>, 'Trainable: False'),
 (<keras.layers.core.Flatten at 0x7f3963894ef0>, 'Trainable: False'),
 (<keras.layers.core.Dropout at 0x7f396386a4a8>, 'Trainable: False'),
 (<keras.layers.core.Dense at 0x7f39638d1dd8>, 'Trainable: True'),
 (<keras.layers.core.Dense at 0x7f3963880fd0>, 'Trainable: True')]

In [8]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit(
    X_train, y_train, epochs=30, 
    batch_size=64, class_weight={0:1, 1:6}, validation_split=0.01,
    callbacks=[early_stopping]
)

Train on 1603 samples, validate on 17 samples
Epoch 1/30
1603/1603 [==============================] - 1s 791us/step - loss: 1.1769 - acc: 0.6201 - val_loss: 0.8828 - val_acc: 0.5294
Epoch 2/30
1603/1603 [==============================] - 1s 619us/step - loss: 1.1389 - acc: 0.6394 - val_loss: 0.8774 - val_acc: 0.5882
Epoch 3/30
1603/1603 [==============================] - 1s 622us/step - loss: 1.1166 - acc: 0.6394 - val_loss: 0.7899 - val_acc: 0.5294
Epoch 4/30
1603/1603 [==============================] - 1s 626us/step - loss: 1.0858 - acc: 0.6675 - val_loss: 0.7747 - val_acc: 0.5882
Epoch 5/30
1603/1603 [==============================] - 1s 629us/step - loss: 1.0464 - acc: 0.6868 - val_loss: 0.7713 - val_acc: 0.6471
Epoch 6/30
1603/1603 [==============================] - 1s 625us/step - loss: 1.0177 - acc: 0.7031 - val_loss: 0.7715 - val_acc: 0.6471
Epoch 7/30
1603/1603 [==============================] - 1s 625us/step - loss: 0.9899 - acc: 0.6968 - val_loss: 0.7450 - val_acc: 0.7059
Ep

In [9]:
get_metrics(model, X_test, y_test)

{'accuracy': 0.69999999999999996,
 'precision': 0.29310344827586204,
 'recall': 0.56666666666666665,
 'roc_auc': 0.72088888888888891}

In [10]:
from os.path import basename
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K

def fix_layers(model, fixed_layers):
    for i in range(fixed_layers):
        model.layers[i].trainable = False
    
    model.compile(loss='binary_crossentropy',
              optimizer='rmsprop', 
              metrics=['accuracy'])
    
def fine_tune(fixed_layers):
    model = load_model("models/model.h5")
    
    fix_layers(model, fixed_layers)
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)

    model.fit(
        X_train, y_train, epochs=10, 
        batch_size=64, class_weight={0:1, 1:6}, validation_split=0.01,
        callbacks=[early_stopping]
    )
    
    return model

def get_analysis(filename, fixed_layers=4):
    K.clear_session()
    model = load_model("models/model.h5")
    X, y = load_data(files[143])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)
    
    ret = {"file" : basename(filename)}
    metrics = {"ft_0_{}".format(k):v for k,v in get_metrics(model, X_test, y_test).items()}
    ret.update(metrics)
    
    model = fine_tune(4)
    metrics = {"ft_4_{}".format(k):v for k,v in get_metrics(model, X_test, y_test).items()}
    ret.update(metrics)
    
    model = fine_tune(5)
    metrics = {"ft_5_{}".format(k):v for k,v in get_metrics(model, X_test, y_test).items()}
    ret.update(metrics)
    
    K.clear_session()
    return ret

get_analysis(files[100], 4)

Reading /home/ubuntu/corpora/sets/PruebasMuseo_7488001.fdt
Reading 0 ... 63231  =      0.000 ...   493.992 secs...
Setting up low-pass filter at 20 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 169 samples (1.320 sec) selected
1800 events found
Events id: [1 2]
1800 matching events found
0 projection items activated
Loading data for 1800 events and 104 original time points ...
0 bad epochs dropped
Train on 1603 samples, validate on 17 samples
Epoch 1/10
1603/1603 [==============================] - 1s 741us/step - loss: 1.1829 - acc: 0.5970 - val_loss: 0.8507 - val_acc: 0.5294
Epoch 2/10
1603/1603 [==============================] - 1s 622us/step - loss: 1.1343 - acc: 0.6351 - val_loss: 0.8631 - val_acc: 0.5294
Epoch 3/10
1603/1603 [==============================] - 1s 625us/step - loss: 1.1150 - acc: 0.6444 - val_loss: 0.8155 - val_acc: 0.5294
Epoch 4/10
1603/1603 [==============================] - 1s 626us/step - loss: 1.0805 - acc: 0.6550 - val_loss: 0.7998 - val_acc: 0.52

{'file': 'PruebasMuseo_28970001.set',
 'ft_0_accuracy': 0.5444444444444444,
 'ft_0_precision': 0.23469387755102042,
 'ft_0_recall': 0.76666666666666672,
 'ft_0_roc_auc': 0.67955555555555558,
 'ft_4_accuracy': 0.71111111111111114,
 'ft_4_precision': 0.35135135135135137,
 'ft_4_recall': 0.8666666666666667,
 'ft_4_roc_auc': 0.8626666666666668,
 'ft_5_accuracy': 0.57222222222222219,
 'ft_5_precision': 0.21686746987951808,
 'ft_5_recall': 0.59999999999999998,
 'ft_5_roc_auc': 0.68600000000000005}

In [11]:
analysis = [get_analysis(file) for file in files[-10:]]
    

Reading /home/ubuntu/corpora/sets/PruebasMuseo_7488001.fdt
Reading 0 ... 63231  =      0.000 ...   493.992 secs...
Setting up low-pass filter at 20 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 169 samples (1.320 sec) selected
1800 events found
Events id: [1 2]
1800 matching events found
0 projection items activated
Loading data for 1800 events and 104 original time points ...
0 bad epochs dropped
Train on 1603 samples, validate on 17 samples
Epoch 1/10
1603/1603 [==============================] - 1s 778us/step - loss: 1.1814 - acc: 0.6220 - val_loss: 0.8698 - val_acc: 0.5294
Epoch 2/10
1603/1603 [==============================] - 1s 621us/step - loss: 1.1409 - acc: 0.6307 - val_loss: 0.8050 - val_acc: 0.7059
Epoch 3/10
1603/1603 [==============================] - 1s 612us/step - loss: 1.1137 - acc: 0.6644 - val_loss: 0.8641 - val_acc: 0.5294
Epoch 4/10
1603/1603 [==============================] - 1s 620us/step - loss: 1.0902 - acc: 0.6263 - val_loss: 0.7780 - val_acc: 0.64

1603/1603 [==============================] - 0s 278us/step - loss: 1.1628 - acc: 0.5783 - val_loss: 0.8974 - val_acc: 0.5294
Epoch 8/10
1603/1603 [==============================] - 0s 281us/step - loss: 1.1565 - acc: 0.5839 - val_loss: 0.8978 - val_acc: 0.4706
Reading /home/ubuntu/corpora/sets/PruebasMuseo_7488001.fdt
Reading 0 ... 63231  =      0.000 ...   493.992 secs...
Setting up low-pass filter at 20 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 169 samples (1.320 sec) selected
1800 events found
Events id: [1 2]
1800 matching events found
0 projection items activated
Loading data for 1800 events and 104 original time points ...
0 bad epochs dropped
Train on 1603 samples, validate on 17 samples
Epoch 1/10
1603/1603 [==============================] - 1s 752us/step - loss: 1.1802 - acc: 0.5976 - val_loss: 0.8515 - val_acc: 0.5294
Epoch 2/10
1603/1603 [==============================] - 1s 623us/step - loss: 1.1569 - acc: 0.6294 - val_loss: 0.8162 - val_acc: 0.5294
Epoch 3/

1603/1603 [==============================] - 0s 279us/step - loss: 1.1601 - acc: 0.6001 - val_loss: 0.9063 - val_acc: 0.5294
Reading /home/ubuntu/corpora/sets/PruebasMuseo_7488001.fdt
Reading 0 ... 63231  =      0.000 ...   493.992 secs...
Setting up low-pass filter at 20 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 169 samples (1.320 sec) selected
1800 events found
Events id: [1 2]
1800 matching events found
0 projection items activated
Loading data for 1800 events and 104 original time points ...
0 bad epochs dropped
Train on 1603 samples, validate on 17 samples
Epoch 1/10
1603/1603 [==============================] - 1s 756us/step - loss: 1.1848 - acc: 0.5964 - val_loss: 0.8515 - val_acc: 0.5294
Epoch 2/10
1603/1603 [==============================] - 1s 616us/step - loss: 1.1380 - acc: 0.6207 - val_loss: 0.8327 - val_acc: 0.5294
Epoch 3/10
1603/1603 [==============================] - 1s 624us/step - loss: 1.0989 - acc: 0.6519 - val_loss: 0.7907 - val_acc: 0.6471
Epoch 4/

1603/1603 [==============================] - 1s 768us/step - loss: 1.1788 - acc: 0.5964 - val_loss: 0.8831 - val_acc: 0.5294
Epoch 2/10
1603/1603 [==============================] - 1s 624us/step - loss: 1.1397 - acc: 0.6357 - val_loss: 0.9249 - val_acc: 0.5294
Epoch 3/10
1603/1603 [==============================] - 1s 619us/step - loss: 1.1051 - acc: 0.6151 - val_loss: 0.7990 - val_acc: 0.5882
Epoch 4/10
1603/1603 [==============================] - 1s 613us/step - loss: 1.0776 - acc: 0.6606 - val_loss: 0.7639 - val_acc: 0.6471
Epoch 5/10
1603/1603 [==============================] - 1s 617us/step - loss: 1.0411 - acc: 0.6837 - val_loss: 0.7776 - val_acc: 0.6471
Epoch 6/10
1603/1603 [==============================] - 1s 623us/step - loss: 1.0279 - acc: 0.6756 - val_loss: 0.7263 - val_acc: 0.7059
Epoch 7/10
1603/1603 [==============================] - 1s 623us/step - loss: 0.9690 - acc: 0.7099 - val_loss: 0.8187 - val_acc: 0.5882
Epoch 8/10
1603/1603 [==============================] - 1s 

In [12]:
import pandas as pd

df = pd.DataFrame(df)

,file,ft_0_accuracy,ft_0_precision,ft_0_recall,ft_0_roc_auc,ft_4_accuracy,ft_4_precision,ft_4_recall,ft_4_roc_auc,ft_5_accuracy,ft_5_precision,ft_5_recall,ft_5_roc_auc
0,PruebasMuseo_21668001.set,0.622222,0.284091,0.833333,0.726889,0.750000,0.377049,0.766667,0.863333,0.627778,0.277108,0.766667,0.732000
1,PruebasMuseo_27157001.set,0.588889,0.244186,0.700000,0.716667,0.800000,0.446429,0.833333,0.889111,0.611111,0.261905,0.733333,0.728000
2,PruebasMuseo_13235001.set,0.550000,0.213483,0.633333,0.649333,0.733333,0.344828,0.666667,0.824444,0.561111,0.211765,0.600000,0.650222
3,PruebasMuseo_31056001.set,0.594444,0.258427,0.766667,0.703111,0.777778,0.395833,0.633333,0.827111,0.616667,0.259259,0.700000,0.688444
4,PruebasMuseo_9809001.set,0.605556,0.253012,0.700000,0.681333,0.850000,0.533333,0.800000,0.906000,0.611111,0.256098,0.700000,0.692222
5,PruebasMuseo_25302001.set,0.566667,0.233333,0.700000,0.682667,0.766667,0.388889,0.700000,0.863556,0.572222,0.235955,0.700000,0.680000
6,PruebasMuseo_18046001.set,0.511111,0.210000,0.700000,0.644444,0.577778,0.265306,0.866667,0.802000,0.561111,0.230769,0.700000,0.650667
7,PruebasMuseo_16943001.set,0.600000,0.255814,0.733333,0.693333,0.755556,0.383333,0.766667,0.848000,0.611111,0.261905,0.733333,0.680222
8,PruebasMuseo_7488001.set,0.538889,0.226804,0.733333,0.679111,0.750000,0.388060,0.866667,0.893778,0.555556,0.228261,0.700000,0.681333
9,PruebasMuseo_12168001.set,0.611111,0.267442,0.766667,0.744222,0.822222,0.477273,0.700000,0.853556,0.611111,0.267442,0.766667,0.754000


In [14]:
df.to_csv()

AttributeError: 'list' object has no attribute 'to_csv'